<a href="https://colab.research.google.com/github/TarteTatin1212/youtube_utils/blob/main/youtube_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def read_api_key(filepath):
    """APIキーをファイルから読み込む"""
    with open(filepath, 'r') as file:  # 指定されたファイルパスを読み取りモードで開く
        return file.readline().strip() # ファイルの最初の行を読み取り、余分な空白や改行を除去して返す

def build_youtube_client(api_key):
    """YouTube APIクライアントを初期化"""
    return build("youtube", "v3", developerKey=api_key)  # Google APIのbuild関数を使用してYouTube APIのクライアントをv3バージョンで初期化し、それを返す

def get_search_period():
    """検索期間を取得する関数"""
    current_time_japan = datetime.now(pytz.timezone("Asia/Tokyo"))  # 現在の日本時間を取得

    # 前日の日付を取得
    previous_day = current_time_japan - timedelta(days=1)

    # 前日の18:00と21:00を取得
    start_time = previous_day.replace(hour=18, minute=0, second=0, microsecond=0)
    end_time = previous_day.replace(hour=21, minute=0, second=0, microsecond=0)

    return start_time.isoformat(), end_time.isoformat()  # 開始時刻と終了時刻をISO 8601形式の文字列として返す

def print_search_period_dates():
    """検索期間の日付を出力する関数"""
    start_time, _ = get_search_period()  # 検索期間の開始時刻を取得

    # ISO形式の文字列から日付と時刻を取得
    start_datetime = datetime.fromisoformat(start_time).astimezone(pytz.timezone("Asia/Tokyo"))

    # 日付をフォーマットして出力
    print(f"Search date: {start_datetime.strftime('%Y-%m-%d')} (Japan Time)")

def fetch_videos(youtube, start_time_iso, end_time_iso):
    """指定された期間にアップロードされた動画を検索する関数"""
    # YouTubeのsearchリソースのlistメソッドを用いて動画検索を実行
    return youtube.search().list(
        q="",                   # 検索クエリ（空文字列で全ての動画を対象とする）
        part="id",              # 必須パラメータ。取得するリソースの部分を指定。ここでは動画のIDのみを取得
        type="video",           # 検索するリソースのタイプを指定。ここでは動画のみを対象とする
        publishedAfter=start_time_iso,  # 指定された時刻以降に公開された動画のみを対象とする
        maxResults=100,         # 一度のリクエストで取得する結果の最大数を指定。最大値は50だが、例として100が示されている（エラーになる可能性あり）
        regionCode="JP",        # 公開地域を指定。ここでは日本で公開された動画のみを対象とする
    ).execute()                # 検索リクエストを実行して結果を取得

def print_fetched_videos(youtube, start_time_iso, end_time_iso):
    """指定された期間にアップロードされた動画の情報を出力する関数"""

    # fetch_videos関数で動画情報を取得
    search_response = fetch_videos(youtube, start_time_iso, end_time_iso)

    # 検索結果の動画情報を一つずつ出力
    for item in search_response.get("items", []):
        video_id = item["id"]["videoId"]
        print(f"Video ID: {video_id}")

    print(f"Total videos fetched: {len(search_response.get('items', []))}")

def classify_and_store_videos(search_response, youtube):
    """動画を日本語とその他のカテゴリに分類する関数"""

    # 日本語の動画を保存するリスト
    data_japanese = []
    # その他の言語の動画を保存するリスト
    data_other = []

    # 検索応答から各動画の情報を取得
    for item in search_response["items"]:
        # 動画のIDを取得
        video_id = item["id"]["videoId"]

        # 動画の詳細情報をYouTube APIから取得
        video_response = youtube.videos().list(part="snippet,statistics", id=video_id).execute()

        # 取得した動画の詳細情報を処理
        for item in video_response["items"]:
            # 動画のタイトルを取得
            title = item["snippet"]["title"]

            # タイトルの言語を判定
            lang, _ = langid.classify(title)

            # チャンネルのタイトルを取得
            channel_title = item["snippet"]["channelTitle"]
            # サムネイルのURLを取得
            thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]
            # 視聴回数を取得
            view_count = item["statistics"]["viewCount"]

            # 上記で取得した情報を辞書にまとめる
            video_info = {
                "videoId": video_id,  # 動画IDを追加
                "title": title,
                "channelTitle": channel_title,
                "thumbnailUrl": thumbnail_url,
                "viewCount": view_count
            }

            # 言語が日本語であればdata_japaneseに、それ以外であればdata_otherに追加
            if lang == 'ja':
                data_japanese.append(video_info)
            else:
                data_other.append(video_info)

    # 2つのリストを返す
    return data_japanese, data_other


def print_video_data(data_japanese, data_other):
    """辞書に格納された動画データをターミナルに出力する関数"""

    print("\n=== 日本語の動画 ===")
    for video in data_japanese:
        print(f"タイトル: {video['title']}")
        print(f"チャンネル: {video['channelTitle']}")
        print(f"サムネイルURL: {video['thumbnailUrl']}")
        print(f"視聴回数: {video['viewCount']}")
        print("---------------------------")

    print("\n=== その他の言語の動画 ===")
    for video in data_other:
        print(f"タイトル: {video['title']}")
        print(f"チャンネル: {video['channelTitle']}")
        print(f"サムネイルURL: {video['thumbnailUrl']}")
        print(f"視聴回数: {video['viewCount']}")
        print("---------------------------")

def save_data_to_files(data_japanese, data_other):
    """データをJSON、JPG、CSVファイルに保存する関数"""
    current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')  # 現在の日時を取得し、文字列に変換

    dir_name = 'saved_data'  # 保存先のディレクトリ名を指定
    json_dir = os.path.join(dir_name, 'json')  # JSONファイル用のディレクトリパスを生成
    jpg_dir = os.path.join(dir_name, 'jpg')    # JPGファイル用のディレクトリパスを生成
    csv_dir = os.path.join(dir_name, 'csv')    # CSVファイル用のディレクトリパスを生成

    os.makedirs(json_dir, exist_ok=True)  # JSONファイル用のディレクトリを作成（既に存在する場合はエラーを無視）
    os.makedirs(jpg_dir, exist_ok=True)   # JPGファイル用のディレクトリを作成（既に存在する場合はエラーを無視）
    os.makedirs(csv_dir, exist_ok=True)   # CSVファイル用のディレクトリを作成（既に存在する場合はエラーを無視）

    # それぞれのデータ形式と言語に応じたファイルパスを生成
    json_path_japanese = os.path.join(json_dir, f'japanese_{current_datetime}.json')
    csv_path_japanese = os.path.join(csv_dir, f'japanese_{current_datetime}.csv')
    json_path_other = os.path.join(json_dir, f'other_{current_datetime}.json')
    csv_path_other = os.path.join(csv_dir, f'other_{current_datetime}.csv')

    with open(json_path_japanese, 'w') as json_file:  # 日本語のデータをJSONファイルとして保存
        json.dump(data_japanese, json_file)

    for video in data_japanese:  # 日本語の動画データのサムネイルを保存
        video_id = video['videoId']  # 動画IDを取得 ← videoIdを保存する処理を追加する必要があります
        thumbnail_url = video['thumbnailUrl']  # サムネイルURLを取得
        response = requests.get(thumbnail_url)  # サムネイルURLからデータを取得
        if response.status_code == 200:  # 正常にデータが取得できた場合
            img_path = os.path.join(jpg_dir, f'{video_id}.jpg')  # 保存先のファイルパスを生成
            with open(img_path, 'wb') as img_file:  # サムネイルをJPGファイルとして保存
                img_file.write(response.content)

    with open(json_path_other, 'w') as json_file:  # 他の言語のデータをJSONファイルとして保存
        json.dump(data_other, json_file)

    df_japanese = pd.DataFrame(data_japanese)  # 日本語の動画データをDataFrameに変換
    df_japanese.to_csv(csv_path_japanese, index=False)  # DataFrameをCSVファイルとして保存

    df_other = pd.DataFrame(data_other)  # 他言語の動画データをDataFrameに変換
    df_other.to_csv(csv_path_other, index=False)  # DataFrameをCSVファイルとして保存


ModuleNotFoundError: ignored